In [1]:
import pandas as pd
import numpy as np

income = "Income.csv"
income_ori = pd.read_csv(income)
income_ori.head()

,Unnamed: 0,State_ab,City,Zip_Code,Lat,Lon,Mean,Median,Stdev
0,0,AL,Chickasaw,36611,30.771450,-88.079697,38773,30506,33101
1,1,AL,Louisville,36048,31.708516,-85.611039,37725,19528,43789
2,2,AL,Columbiana,35051,33.191452,-86.615618,54606,31930,57348
3,3,AL,Satsuma,36572,30.874343,-88.009442,63919,52814,47707
4,4,AL,Dauphin Island,36528,30.250913,-88.171268,77948,67225,54270


In [2]:
# we only keep several coloums: 
income_df = income_ori[["Zip_Code","Mean","Median","Stdev","Lat","Lon"]]
income_df.head()

,Zip_Code,Mean,Median,Stdev,Lat,Lon
0,36611,38773,30506,33101,30.771450,-88.079697
1,36048,37725,19528,43789,31.708516,-85.611039
2,35051,54606,31930,57348,33.191452,-86.615618
3,36572,63919,52814,47707,30.874343,-88.009442
4,36528,77948,67225,54270,30.250913,-88.171268


In [3]:
# rename the column
income_df = income_df.rename(columns = {"Zip_Code" : "zip_code1","Mean":"mean","Median":"median","Stdev":"stdev","Lat":"lat","Lon":"lon"})
income_df.head()

,zip_code1,mean,median,stdev,lat,lon
0,36611,38773,30506,33101,30.771450,-88.079697
1,36048,37725,19528,43789,31.708516,-85.611039
2,35051,54606,31930,57348,33.191452,-86.615618
3,36572,63919,52814,47707,30.874343,-88.009442
4,36528,77948,67225,54270,30.250913,-88.171268


In [4]:
# show the first 10 duplicates
income_df["zip_code"] = income_df["zip_code1"].apply(lambda x: str(x).zfill(5))
income_df["zip_code"].value_counts().sort_values(ascending=False).head(10)

94606    19
11203    16
11204    15
11377    15
92626    15
88001    14
00928    14
11206    13
78584    13
79760    13
Name: zip_code, dtype: int64

In [5]:
# drop the orginal 9-digit zipcode
income_df = income_df.drop(["zip_code1"], axis=1)

In [6]:
income_df.head()

,mean,median,stdev,lat,lon,zip_code
0,38773,30506,33101,30.771450,-88.079697,36611
1,37725,19528,43789,31.708516,-85.611039,36048
2,54606,31930,57348,33.191452,-86.615618,35051
3,63919,52814,47707,30.874343,-88.009442,36572
4,77948,67225,54270,30.250913,-88.171268,36528


In [7]:
income_df["zip_code"].value_counts().sort_values(ascending=False).head(10)

94606    19
11203    16
11204    15
11377    15
92626    15
88001    14
00928    14
11206    13
78584    13
79760    13
Name: zip_code, dtype: int64

In [8]:
#income_df_duplicate = income_df.loc[income_df["zip_code"]=="00928"]
income_df_groupby = income_df.groupby(["zip_code"])
income_df_ave = income_df_groupby.mean()
income_df_ave.head()

,mean,median,stdev,lat,lon
zip_code,,,,,
00601,22496.333333,13144.333333,27148.666667,18.171777,-66.713307
00602,15078.400000,11323.600000,12503.600000,18.384709,-67.193233
00603,26046.800000,15386.400000,29969.600000,18.439458,-67.123043
00605,27497.500000,9461.000000,41453.500000,18.472294,-67.129718
00606,19194.000000,15233.000000,15261.000000,18.192187,-66.980897


In [19]:
#drop the missing data
clean_mean = []
for i in income_df_ave["mean"]:
    if i == 0:
        i = np.nan
    clean_mean.append(i)
        
income_df_ave["clean_mean"] = clean_mean
in_df_ave_clean_mean = income_df_ave.dropna(how = "any")

# drop the orginal mean
in_df_ave_clean_mean2 = in_df_ave_clean_mean.drop(["mean"], axis=1)

In [20]:
in_df_ave_clean_mean2["zip_code"] = in_df_ave_clean_mean2.index
in_df_ave_float = in_df_ave_clean_mean2.reset_index(drop=True)
in_df_final = in_df_ave_float.round({"clean_mean":2,"median":2,"stdev":2,"lat":4,"lon":4})
in_df_final.head()

,median,stdev,lat,lon,clean_mean,zip_code
0,13144.33,27148.67,18.1718,-66.7133,22496.33,00601
1,11323.60,12503.60,18.3847,-67.1932,15078.40,00602
2,15386.40,29969.60,18.4395,-67.1230,26046.80,00603
3,9461.00,41453.50,18.4723,-67.1297,27497.50,00605
4,15233.00,15261.00,18.1922,-66.9809,19194.00,00606


In [22]:
# save data into csv
in_df_final.to_csv("cleaned_income2.csv", sep=',', encoding='utf-8', index=False)